<a href="https://colab.research.google.com/github/mkrause6/hello/blob/master/CSC8490MatthewKrause.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up Oracle client

! pip install cx_Oracle

! ls -l oracle-instantclient*-basiclite-*.rpm || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

! ls -l oracle-instantclient*-sqlplus-*.rpm   || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

! sudo apt-get install alien libaio1

! sudo alien -i oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

! sudo alien -i oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

%env ORACLE_HOME=/usr/lib/oracle/19.3/client64

%env LD_LIBRARY_PATH=/usr/lib/oracle/19.3/client64/lib

print('Setup finished!')

     |████████████████████████████████| 757kB 4.7MB/s 
ls: cannot access 'oracle-instantclient*-basiclite-*.rpm': No such file or directory
--2020-11-25 03:06:42--  https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm
Resolving yum.oracle.com (yum.oracle.com)... 23.76.88.244
Connecting to yum.oracle.com (yum.oracle.com)|23.76.88.244|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28553496 (27M) [application/x-rpm]
Saving to: ‘oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm’

oracle-instantclien 100%[===================>]  27.23M   141MB/s    in 0.2s    

2020-11-25 03:06:42 (141 MB/s) - ‘oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm’ saved [28553496/28553496]

ls: cannot access 'oracle-instantclient*-sqlplus-*.rpm': No such file or directory
--2020-11-25 03:06:43--  https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/o

In [2]:
# Create Oracle connection and cursor
import numpy as np
import pandas as pd
import cx_Oracle

CONN_INFO = {
 'host': '45.79.135.253',
 'port': '1521',
 'user': 'username',
 'psw': 'password',
 'service': 'xe',
}
CONN_STR = '{user}/{psw}@{host}:{port}/{service}'.format(**CONN_INFO)
connection = cx_Oracle.connect(CONN_STR)

cursor = connection.cursor()

print(connection.version)

18.4.0.0.0


In [4]:
# Get a general report for the TV show Chuck
show_id = "000000000"
show_info = []

for row in cursor.execute(f"SELECT * FROM TVShow WHERE ShowId = {show_id}"):
    show_info.append(row)

print(f"Show Title: {show_info[0][4]}")
print(f"Genre: {show_info[0][6]}")
print(f"Number of Episodes: {show_info[0][5]}")
print(f"Release Date: {show_info[0][7].strftime('%m-%d-%Y')}")

roles = []
for row in cursor.execute(f"SELECT * FROM TVRole WHERE ShowId = {show_id}"):
    roles.append(row)

actors = []
for role in roles:
    for row in cursor.execute(f"SELECT * FROM Actor WHERE ActorId = {role[0]}"):
        actors.append(row[1])

print("\nCast: ")
for actor, role in zip(actors, roles):
  print(f"\t{actor}  -  {role[2]}")

Show Title: Chuck
Genre: Action
Number of Episodes: 91
Release Date: 09-24-2007

Cast: 
	Zachary Levi  -  Charles Irving Bartowski
	Yvonne Strahovski  -  Sarah Walker


In [5]:
# Insert basic information on the Netflix original movie Bird Box 
cursor.execute("INSERT INTO Producer (ProducerId, Name, Birthdate, Hometown, Writer) VALUES (:1, :2, :3, :4, :5)",
               ('00000009', 'Chris Morgan', '24-Nov-1966', 'Chicago, IL', 'Y'))
cursor.execute("INSERT INTO Actor (ActorId, Name, Birthdate, Hometown, Alister) VALUES (:1, :2, :3, :4, :5)",
               ('0000000010', 'Sandra Bullock', '26-Jul-1964', 'Arlington, VA', 'Y'))
cursor.execute("INSERT INTO Movie (MovieId, ProducerId, Name, Genre, ReleaseDate, Runtime) VALUES (:1, :2, :3, :4, :5, :6)",
               ('000000003', '00000009', 'Bird Box', 'Thriller', '14-Dec-2018', '124'))
cursor.execute("INSERT INTO MovieRole (ActorId, MovieId, CharacterName, LeadCharacter, ActingType, ScreenTime) VALUES (:1, :2, :3, :4, :5, :6)",
               ('0000000010', '000000003', 'Malorie Hayes', 'Y', 'Normal', None))
cursor.execute("INSERT INTO StreamMovie (MovieId, ServiceId) VALUES (:1, :2)",
               ('000000003', '0000000'))

In [6]:
# Print movie title with "starring" string
movie_id = '000000003'
movie_title = []

for row in cursor.execute(f"SELECT * FROM Movie WHERE MovieId = {movie_id}"):
    movie_title.append(row[2])

# Setup variable to store the result of the procedure and call the procedure
result = cursor.var(cx_Oracle.STRING)
cursor.callproc("GetStarringString", [movie_id, result])

print(movie_title[0])
print("   " + result.getvalue())

Bird Box
   Starring Sandra Bullock


In [7]:
# Add an actor and his role (Trevante Rhodes - Tom) to the Bird Box information
cursor.execute("INSERT INTO Actor (ActorId, Name, Birthdate, Hometown, Alister) VALUES (:1, :2, :3, :4, :5)",
               ('0000000011', 'Trevante Rhodes', '10-Feb-1990', 'Ponchatoula, LA', 'N'))
cursor.execute("INSERT INTO MovieRole (ActorId, MovieId, CharacterName, LeadCharacter, ActingType, ScreenTime) VALUES (:1, :2, :3, :4, :5, :6)",
               ('0000000011', '000000003', 'Tom', 'Y', 'Normal', None))

In [8]:
# Print movie title with updated "starring" string
result = cursor.var(cx_Oracle.STRING)
cursor.callproc("GetStarringString", [movie_id, result])

print(movie_title[0])
print("   " + result.getvalue())

Bird Box
   Starring Sandra Bullock, Trevante Rhodes


In [9]:
# Make changes public and permanent
connection.commit()

In [10]:
connection.close()